# Chapter 3: Rust's Cryptographic Strengths and Vulnerabilities

## Introduction

In this chapter, our exploration into Rust's cryptographic landscape reveals a canvas of libraries and tools designed to strengthen digital communications. At the forefront stands the [`rustls`](https://github.com/rustls/rustls) library, a robust Rust implementation of TLS, which serves as the cornerstone for securing SSL/TLS communications. Configuring a secure server involves creating an instance of [`ServerConfig`](https://docs.rs/rustls/latest/rustls/server/struct.ServerConfig.html) and tailoring it to specific needs, such as cipher suite preferences and key management. Rust's type safety and expressive syntax contribute to a more resilient implementation, mitigating common pitfalls associated with memory safety.

Beyond the SSL/TLS domain, Rust empowers us to tackle the complexities of [mutual authentication](https://en.wikipedia.org/wiki/Mutual_authentication) using the [`openssl`](https://en.wikipedia.org/wiki/OpenSSL) library. This involves crafting a secure handshake process where both client and server authenticate each other's identity through [X.509](https://en.wikipedia.org/wiki/X.509) certificates. Configuring an SSL acceptor involves loading server certificates and private keys while establishing trust relationships through [CA certificates](https://en.wikipedia.org/wiki/Certificate_authority). Rust's emphasis on [ownership](https://doc.rust-lang.org/book/ch04-00-understanding-ownership.html) and [lifetimes](https://doc.rust-lang.org/book/ch10-00-generics.html#generic-types-traits-and-lifetimes) ensures that [cryptographic keys](https://en.wikipedia.org/wiki/Key_(cryptography)) and sensitive data are managed securely throughout the authentication process, reducing the risk of memory leaks and unauthorized access.

As we navigate Rust's cryptographic terrain, [symmetric-key cryptography](https://en.wikipedia.org/wiki/Symmetric-key_algorithm) emerges as a pivotal aspect, facilitated by libraries like [`crypto`](https://crates.io/crates/crypto). Our exploration takes us into the domain of [the Advanced Encryption Standard (AES)](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard), where data [encryption](https://en.wikipedia.org/wiki/Encryption) and [decryption](https://en.wikipedia.org/wiki/Encryption) become intricate dances with key sizes and block modes. [Rust's focus on zero-cost abstractions](https://doc.rust-lang.org/beta/embedded-book/static-guarantees/zero-cost-abstractions.html) and performance ensures that cryptographic operations are executed efficiently, catering to the demands of secure data transmission. Leveraging Rust's [concurrency model](https://doc.rust-lang.org/book/ch16-00-concurrency.html), we can parallelize cryptographic tasks, enhancing throughput without compromising security.

## Table of Contents

* TODO

## 1. Cryptography in Rust

Before kicking off our exploration of cryptographic operations in Rust, it's essential to immerse ourselves in the fundamental concepts that make this complex field. We'll traverse these concepts meticulously to ensure a robust comprehension of the cryptographic landscape.

### 1.1 Encryption and Decryption

[Encryption](https://en.wikipedia.org/wiki/Encryption), a part of cryptography, surpasses mere confidentiality maintenance. It represents a dual-purpose functionality, allowing data scrambling and subsequent unscrambling. At its core, encryption involves a cryptographic function, that takes both data and a key as inputs, yielding either ciphertext or the original cleartext. [Symmetric algorithms](https://en.wikipedia.org/wiki/Symmetric-key_algorithm) work with a single key for both encryption and decryption, while [asymmetric](https://en.wikipedia.org/wiki/Public-key_cryptography) counterparts work with different keys.

At the heart of our cryptographic exploration lies [ChaCha20Poly1305](https://en.wikipedia.org/wiki/ChaCha20-Poly1305) using the [`chacha20poly1305`](https://docs.rs/chacha20poly1305/latest/chacha20poly1305/index.html) crate, a cipher notable for its simplicity and speed. The underlying [ChaCha20](https://en.wikipedia.org/wiki/Salsa20#ChaCha_variant) stream cipher, employing a blend of add, rotate, and XOR instructions (ARX), works seamlessly with the straightforward [Poly1305](https://en.wikipedia.org/wiki/Poly1305) hash function. While not universally endorsed by standards bodies like [NIST](https://en.wikipedia.org/wiki/National_Institute_of_Standards_and_Technology), ChaCha20Poly1305 is widely adopted, being mandatory in the [Transport Layer Security (TLS)](https://en.wikipedia.org/wiki/Transport_Layer_Security) protocol.

```rust
use chacha20poly1305::{aead::{Aead, KeyInit, OsRng}, ChaCha20Poly1305, Nonce, AeadCore};
use base64::{Engine as _, engine::{self, general_purpose}, alphabet};
{
    let key = ChaCha20Poly1305::generate_key(&mut OsRng);
    let cipher = ChaCha20Poly1305::new(&key);
    let nonce = ChaCha20Poly1305::generate_nonce(&mut OsRng);
    let plaintext = "Hello, Rust Cryptography!";
    
    // Encryption
    let ciphertext = cipher.encrypt(&nonce, plaintext.as_bytes())?;
    let b64_ciphertext = general_purpose::STANDARD.encode(&ciphertext);
    println!("Base64 Cipher Text: {}", b64_ciphertext);

    // Decryption
    let decrypted_text = cipher.decrypt(&nonce, &*ciphertext)?;
    println!("Decrypted Text: {}", String::from_utf8_lossy(&decrypted_text));
}
```

Encryption works like a protector for information moving from one place to another and a keeper of important data stored away. It waits to be unlocked for later use or carefully watched for any signs of deceitful actions.

In [2]:
:dep chacha20poly1305 = {version="0.10.1"}

In [3]:
:dep base64 = {version="0.21.5"}

In [4]:
use chacha20poly1305::{aead::{Aead, KeyInit, OsRng}, ChaCha20Poly1305, Nonce, AeadCore};
use base64::{Engine as _, engine::{self, general_purpose}, alphabet};
{
    let key = ChaCha20Poly1305::generate_key(&mut OsRng);
    let cipher = ChaCha20Poly1305::new(&key);
    let nonce = ChaCha20Poly1305::generate_nonce(&mut OsRng);
    let plaintext = "Hello, Rust Cryptography!";
    
    // Encryption
    let ciphertext = cipher.encrypt(&nonce, plaintext.as_bytes())?;
    let b64_ciphertext = general_purpose::STANDARD.encode(&ciphertext);
    println!("Base64 Cipher Text: {}", b64_ciphertext);

    // Decryption
    let decrypted_text = cipher.decrypt(&nonce, &*ciphertext)?;
    println!("Decrypted Text: {}", String::from_utf8_lossy(&decrypted_text));
}

Base64 Cipher Text: 5wLvJxUKfPd5bnQWiq2aRtLLIiluVXj7VQkJ8ePOmarTX0v59EOKtRA=
Decrypted Text: Hello, Rust Cryptography!


()

Note that the coded message is printed as a string encoded in base64, and the decoded message is displayed as a standard UTF-8 string. 

### 1.2 Hashing

[Hashing](https://en.wikipedia.org/wiki/Hash_function) stands as a fundamental cryptographic process, operating as a unidirectional function meticulously crafted to produce a fixed-length and inherently unique output, all delegation upon a variable-length input. Its complex elegance lies in the irreversible nature of the transformation, blocking any possibility of figuring out the original input from the resultant hash value. This cryptographic technique finds its power in various scenarios, particularly those where the preservation of the original cleartext source becomes unnecessary for subsequent processing or to guarantee the integrity of data. An exemplary manifestation of secure practices within the domain of information protection is the storage of hashed passwords, a practice ideally complemented by the introduction of [salt](https://en.wikipedia.org/wiki/Salt_(cryptography)) - an additional layer of randomized data. This cryptographic seasoning enhances the unpredictability of the hash values, strengthening security and aligning with best practices in safeguarding sensitive information.

In our exploration of hashing within the Rust programming language, we delve into two illustrative examples. The first demonstrates the cracking of MD5 hashes through an offline [dictionary attack](https://en.wikipedia.org/wiki/Dictionary_attack) using the [`md-5`](https://docs.rs/md-5) crate, employing Rust's capabilities to generate and compare hash values systematically. The second example showcases the implementation of [bcrypt](https://en.wikipedia.org/wiki/Bcrypt) using the [`bcrypt`](https://github.com/Keats/rust-bcrypt) crate, an advanced algorithm enhancing the security of sensitive data such as passwords. Rust's versatility shines as it seamlessly integrates these cryptographic techniques, emphasizing its adaptability in addressing diverse security challenges.

#### 1.2.1 Cracking MD5 Hashes

Let's examine the code for cracking MD5 hashes in Rust:

```rust
use md5::{Md5, Digest};
use hex_literal::hex;

let md5_hash = hex!("517e90f2a52e730701a5d7ec89ef0f40");

let wordlist = vec!["p@ssword231", "12345656789", "password", "Password", "Mahmoud123"];

for password in wordlist {
    let mut md5_hasher = Md5::new();
    md5_hasher.update(password);
    let md5_generated = md5_hasher.finalize();
    let md5_hex_string: String = md5_generated.iter().map(|byte| format!("{:02x}", byte)).collect();

    println!("[INFO] Trying hash {:?}", md5_hex_string);
    if md5_generated[..] == md5_hash {
        println!("[INFO] Password found (MD5): {}", password);
    }
}
```

In this Rust example, we start by defining variable `md5_hash` that hold the target hash value. This hash were obtained post-exploitation, and the goal is to discover the cleartext passwords that produced it. The code reads from a vector of strings, generating MD5 hashes for each password and comparing them with the target hash.

In [5]:
:dep md-5 = {version="0.10.6"}

In [6]:
:dep hex-literal = {version="0.4.1"}

In [7]:
use md5::{Md5, Digest};
use hex_literal::hex;

let md5_hash = hex!("517e90f2a52e730701a5d7ec89ef0f40");

let wordlist = vec!["p@ssword231", "12345656789", "password", "Password", "Mahmoud123"];

for password in wordlist {
    let mut md5_hasher = Md5::new();
    md5_hasher.update(password);
    let md5_generated = md5_hasher.finalize();
    let md5_hex_string: String = md5_generated.iter().map(|byte| format!("{:02x}", byte)).collect();

    println!("[INFO] Trying hash {:?}", md5_hex_string);
    if md5_generated[..] == md5_hash {
        println!("[INFO] Password found (MD5): {}", password);
    }
}

[INFO] Trying hash "ab2d83f72c018c04bda54551f7963e30"
[INFO] Trying hash "576333879d388b3537287481aa07f7c6"
[INFO] Trying hash "5f4dcc3b5aa765d61d8327deb882cf99"
[INFO] Trying hash "dc647eb65e6711e155375218212b3964"
[INFO] Trying hash "517e90f2a52e730701a5d7ec89ef0f40"
[INFO] Password found (MD5): Mahmoud123


()

#### 1.2.2 Implementing bcrypt

Now, let's explore how to use bcrypt to encrypt and authenticate passwords in Rust:

```rust
use bcrypt::{DEFAULT_COST, hash, verify};

let stored_hash = "$2b$12$gPxgyRNV5G/DTaADM4rnuu3LcEbQeVdqhdNaKobgmdiNeyNRmV2me";

let password = "p@ssword123";

// Hash the password using bcrypt
let hashed = hash(password, DEFAULT_COST)?;

println!("Generated Hash: {}", hashed);

// Verify the hashed password against the stored hash
if verify(password, &stored_hash)? {
    println!("[INFO] Authentication successful");
} else {
    println!("[INFO] Authentication failed");
}
```

In this Rust example, we utilize the [`bcrypt`](https://docs.rs/bcrypt) crate to implement bcrypt hashing and authentication. The [`hash`](https://docs.rs/bcrypt/latest/bcrypt/fn.hash.html) function generates a bcrypt hash from a cleartext password, and [`verify`](https://docs.rs/bcrypt/latest/bcrypt/fn.verify.html) is used to compare the generated hash with a stored hash to authenticate the password. The cost factor ensures the algorithm's resource-intensive nature, enhancing security against brute-force attacks.

In [8]:
:dep bcrypt = {version="0.15.0"}

In [9]:
use bcrypt::{DEFAULT_COST, hash, verify};

let stored_hash = "$2b$12$gPxgyRNV5G/DTaADM4rnuu3LcEbQeVdqhdNaKobgmdiNeyNRmV2me";

let password = "p@ssword123";

// Hash the password using bcrypt
let hashed = hash(password, DEFAULT_COST)?;

println!("Generated Hash: {}", hashed);

// Verify the hashed password against the stored hash
if verify(password, &stored_hash)? {
    println!("[INFO] Authentication successful");
} else {
    println!("[INFO] Authentication failed");
}

Generated Hash: $2b$12$BMTC/ZrVK8EYpLWDBm9R/uSG.K1hRUhs8QIGt60XqYmE3QCuvki2q
[INFO] Authentication successful


()

These Rust examples provide insights into the practical application of hashing and bcrypt in real-world scenarios. As we delve into Rust's cryptographic features, a deeper understanding of these concepts will empower your cryptographic journey.

---